In [2]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("하이닉스 power (3).csv", thousands=',', encoding='cp949')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x >= 0.82 else 1 if x >= -0.81 else 0)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_8_features = [feature for feature, importance in features_rf_sorted[:7]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_8_features]
y = df['target']  # y 값을 0과 1로 조정

# # 데이터를 학습 세트와 테스트 세트로 분할
# # Define y as befor
# y = df['target']
# X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=66)

X_train = X_top8.iloc[:1720]
X_test = X_top8.iloc[1720:]
    
y_train = y.iloc[:1720]
y_test = y.iloc[1720:]




# 모델 초기화
models = {
    'SVC': SVC(max_iter=10000,random_state=1),
    'LogisticRegression': LogisticRegression(random_state = 1),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state =1),
    'RandomForestClassifier': RandomForestClassifier(random_state =1),
    'GradientBoostingClassifer' : GradientBoostingClassifier(random_state=1)
}

# 점수를 저장할 딕셔너리
scores = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1_Score': []
}

# 각 모델에 대해 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    scores['Model'].append(model_name)
    scores['Accuracy'].append(accuracy_score(y_test, y_pred))
    scores['Precision'].append(precision_score(y_test, y_pred, average='macro'))
    scores['Recall'].append(recall_score(y_test, y_pred, average='macro'))
    scores['F1_Score'].append(f1_score(y_test, y_pred, average='macro'))

# 점수 딕셔너리를 데이터프레임으로 변환
scores_df = pd.DataFrame(scores)
scores_df

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Model,Accuracy,Precision,Recall,F1_Score
0,SVC,0.353659,0.354913,0.355995,0.353892
1,LogisticRegression,0.329268,0.311974,0.319946,0.298299
2,DecisionTreeClassifier,0.322493,0.323784,0.325162,0.322495
3,RandomForestClassifier,0.356369,0.359711,0.358375,0.356282
4,GradientBoostingClassifer,0.375339,0.380276,0.376945,0.373988


In [3]:
top_8_features

['[일]연기금 등_순매수',
 '고가_대비_종가_등락률',
 '[일]사모펀드_순매수',
 '[일]투신_순매수',
 '[일]은행_순매수',
 '거래량_등락률',
 '[일]프로그램_순매수(종목)']